In [ ]:
import os
import librosa
from datasets import load_dataset, DatasetDict

## Load dataset

In [ ]:
dataset_config = {
  "LOADING_SCRIPT_FILES":  "crema.py",
  "CONFIG_NAME": "clean",
  "DATA_DIR": "data/archive.zip",
  "CACHE_DIR": "cache_crema",
}
ds = load_dataset(
  dataset_config["LOADING_SCRIPT_FILES"],
  dataset_config["CONFIG_NAME"],
  data_dir=dataset_config["DATA_DIR"],
  cache_dir=dataset_config["CACHE_DIR"]
)

## Train test split

In [ ]:
# INTRODUCE TRAIN TEST VAL SPLITS
# 90% train, 10% test + validation
train_testvalid = ds["train"].train_test_split(shuffle=True, test_size=0.1)
# Split the 10% test + valid in half test, half valid
test_valid = train_testvalid["test"].train_test_split(test_size=0.5)
# gather everything into a single DatasetDict
ds = DatasetDict({
     "train": train_testvalid["train"],
     "test": test_valid["test"],
     "val": test_valid["train"],
      }
)

## Convert raw audio into arrays

In [ ]:
# CONVERING RAW AUDIO TO ARRAYS
ds = ds.map( lambda x: {
        "array": librosa.load(x["file"],
         sr=16000,
         mono=False)[0]
          }
)

## Convert label into ids

In [ ]:
ds = ds.class_encode_column("label")

# Selecting subset of dataset

In [ ]:
ds["train"] = ds["train"].select(range(50))